# General queries: no context-based restrictions

In [2]:
from pandas import *
from queryWikidata import query_wikidata

In [3]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## 1. Count all the derivative works present on Wikidata

To do that, we rely on the following properties:
- P737 influenced by
- P144 based on
- P941 inspired by
- P8371 references work
- P4969 derivative work
- P58 screenwriter
- P1877 after a work by
- P674 characters
- P170 creator
- P50 author



In [8]:
q_derivative_works = """
SELECT (COUNT(DISTINCT ?work)AS ?worksCount) (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)

WHERE {
    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
    }
}
"""
derivative_works_df = query_wikidata(endpoint, q_derivative_works, user_agent, True)
derivative_works_df

,worksCount,derivativeWorksCount
0,176364,44378


## 2. Analyse the use of single properties to connect a product with its derivative work

### 2.1 Influenced by (`P737`)

In [15]:
q_influenced_by = """
SELECT DISTINCT
(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)

WHERE {
    ?person wdt:P31 wd:Q5 .
    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df = query_wikidata(endpoint, q_influenced_by, user_agent, True)
influenced_by_df

,work,derivativeWorkInfluencedBy
0,403,674


In [ ]:
""" SELECT DISTINCT
(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
(COUNT(DISTINCT ?influencedWork) AS ?influencedByP) (COUNT(DISTINCT ?workBasedOn) AS ?basedOnP) (COUNT(DISTINCT ?referencedWork) AS ?referencesWorkP) (COUNT(DISTINCT ?derivedWork) AS ?derivedWorkP) (COUNT(DISTINCT ?inspiredWork) AS ?inspiredByP)  (COUNT(DISTINCT ?workAfter) AS ?afterWorkByP)

WHERE {
    ?person wdt:P31 wd:Q5 .
    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }

    {
      ?derivativeWork wdt:P737 ?work . 
      BIND(?derivativeWork AS ?influencedWork)
    }UNION{
      ?derivativeWork wdt:P144 ?work .
      BIND(?derivativeWork AS ?workBasedOn)
    }UNION{
      ?derivativeWork wdt:P941 ?work .
      BIND(?derivativeWork AS ?inspiredWork)
    }UNION{  
      ?derivativeWork wdt:P8371 ?work .
      BIND(?derivativeWork AS ?referencedWork)
    }UNION{
      ?work wdt:P4969 ?derivativeWork .
      BIND(?derivativeWork AS ?derivedWork)
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
      BIND(?derivativeWork AS ?workAfter)
    }
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """